### 1. 환경 설정

In [121]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import re

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

time.sleep(0.2)

### 2. 역대 박스오피스 불러오기(1~100)

In [122]:
data = pd.read_csv('KOBIS_TOP100.csv')
m_list = data['영화명'].astype('string').tolist()
m_year = data['링크연도'].astype('string').tolist()

In [123]:
m_year[6] = '2014'      # 다음에서 베테랑 
m_year[14] = '2018'     # 다음에서 신과함께 - 인과연
m_year[34] = '2013'     # 다음에서 수상한 그녀
m_year[74] = '2014'     # 다음에서 킹스맨:시크릿 에이전트

### 3. 함수 설정

In [124]:
# 평점 더 보기 클릭 
def review_more_click(more_num):
    more_cnt = 0
    
    while True:
        driver.find_element(By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_box > div.alex_more > button').click()
        
        time.sleep(0.5)
        
        more_cnt += 1
        if more_cnt == more_num:
            break

### 4. 영화마다 자동으로 리뷰 읽어오기

In [126]:
%%time
# 다음 영화 사이트 열기
driver.get('https://movie.daum.net/main/')

review_num = 50
more_num = 18     
review_lst = []
movie_rank = 0
is_first = 1

for movie, year in zip(m_list[:10], m_year[:10]):
    # 위로 올라가기 클릭
    driver.find_element(By.CLASS_NAME, 'inner_top').click()
    
    # 검색어 입력
    print(f'--- 영화 {movie}({year})를 검색합니다.', end=' ')
    
    if is_first == 1:
        input_box = driver.find_element(By.CSS_SELECTOR, 'input.tf_keyword')
    else: 
        driver.find_element(By.CLASS_NAME, 'link_search').click()
        input_box = driver.find_element(By.CSS_SELECTOR, 'body > div.kakao_wrap.detail_type > header > div.kakaohead_menu > div > div > form > fieldset > div > input')

    input_box.send_keys(movie)            # 입력
    input_box.send_keys(Keys.ENTER)         # Enter
    # input_box.clear()                     # 입력된 내용 지우기
    time.sleep(0.5)
    
    # 해당 영화 사이트로 이동하기
    driver.find_element(By.CLASS_NAME, 'movie').click()             # 영화 탭 클릭
    time.sleep(0.5)
    
    search_lst = driver.find_elements(By.CSS_SELECTOR, '#mainContent > div > div.box_searchinfo > div.detail_searchinfo.movie.search_result > ul > li')
    keyword = re.sub('[^0-9가-힣]', '', str(movie))       # 영화명 공백 및 특수문자 제거
    for search in search_lst:
        title_link = search.find_element(By.CSS_SELECTOR, 'div > div > strong > a')
        title = re.sub('[^0-9가-힣]', "", title_link.text.split('(')[0])
        # print(title)
        title_year = search.find_element(By.CSS_SELECTOR, 'div > div > dl > dd').text
        # print(title_year, type(title_year), year, type(year))
        if (title == keyword) & (title_year == year):
            # print([title, year])
            title_link.click()
            print(f'>>> 해당 페이지로 이동합니다.')
            break
    time.sleep(0.5)
    
    # 평점 탭 클릭
    sel = 'div.tabmenu_wrap > ul > li > a > span'
    driver.find_elements(By.CSS_SELECTOR, sel)[3].click()
    time.sleep(0.5)

    # 정렬 '최신순' 선택하기
    sel2 = '#alex-area > div > div > div > div.cmt_box > ul.list_category > li:nth-child(3) > button'
    driver.find_element(By.CSS_SELECTOR, sel2).click()
    time.sleep(0.5)
    
    # 평점 더보기 클릭하기
    review_more_click(more_num)
    
    # 한 페이지의 리뷰 전체 불러오기
    sel3 = '#alex-area > div > div > div > div.cmt_box > ul.list_comment > li'
    reviews = driver.find_elements(By.CSS_SELECTOR, sel3)

    review_cnt = 0
    print('Start!', end='')    
    for review in reviews:
        id = review.find_element(By.CSS_SELECTOR, 'div > strong > span > a').text
        
        # 중복인 게시글 건너뛰기
        if is_first == 0:
            if id == review_lst[-1][1]:
                continue
        else:
            pass
        
        # 평점 읽어오기 
        score = review.find_element(By.CSS_SELECTOR, 'div > div').text
        
        # 리뷰 읽어오기
        try:
            text = review.find_element(By.CSS_SELECTOR, 'div > p').text
            if len(text) == 1:          # 성의 없는 댓글(.)은 읽지 않기 
                continue
        except:
            continue
        
        # 데이터셋 만들기 
        review_lst.append([movie, text, score])
        review_cnt += 1
        
        # 메세지 출력
        if review_cnt % 10 == 0:
            if review_cnt != 10 and review_cnt % 100 == 10:
                print(f'\n> {review_cnt}', end=' ')
            else:
                print(f'> {review_cnt}', end=' ')
        
        if review_cnt == review_num:
            print('> END!')
            break
        
    # 영화 하나 끝남 메시지 출력
    is_first = 0
    movie_rank += 1
    print(f'박스오피스 랭킹 {movie_rank}위. {movie}의 리뷰 {review_cnt}개를 모두 수집했습니다.')
    print('-'*80)

--- 영화 명량(2014)를 검색합니다. >>> 해당 페이지로 이동합니다.
Start!> 10 > 20 > 30 > 40 > 50 > END!
박스오피스 랭킹 1위. 명량의 리뷰 50개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 극한직업(2018)를 검색합니다. 

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:" class="link_search">...</a> is not clickable at point (1493, 87). Other element would receive the click: <ul class="list_tabmenu" role="tablist" data-tiara-layer="tab">...</ul>
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x004FF243]
	(No symbol) [0x00487FD1]
	(No symbol) [0x0037D04D]
	(No symbol) [0x003B28B9]
	(No symbol) [0x003B08CC]
	(No symbol) [0x003AE4CB]
	(No symbol) [0x003AD164]
	(No symbol) [0x003A32A6]
	(No symbol) [0x003C858C]
	(No symbol) [0x003A2BFF]
	(No symbol) [0x003C8804]
	(No symbol) [0x003DC9EB]
	(No symbol) [0x003C8386]
	(No symbol) [0x003A163C]
	(No symbol) [0x003A269D]
	GetHandleVerifier [0x00799A22+2655074]
	GetHandleVerifier [0x0078CA24+2601828]
	GetHandleVerifier [0x005A8C0A+619850]
	GetHandleVerifier [0x005A7830+614768]
	(No symbol) [0x004905FC]
	(No symbol) [0x00495968]
	(No symbol) [0x00495A55]
	(No symbol) [0x004A051B]
	BaseThr